In [28]:
# Treasury Forecasting algo version #2 determining the best investnent windows
import mysql.connector
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv(verbose=True)

# Database connection parameters
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME"),
}

print('Database URL: ' + os.environ.get('DB_HOST'))

Database URL: timetables.mysql.database.azure.com


In [29]:
# Fetch data from database and return as a DataFrame
def fetch_data(table_name, column_names='*', condition='1', sql=False):
    try:
        # Connect
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        # Fetch
        if sql:
            cursor.execute(sql)
        else:
            query = f"SELECT {column_names} FROM {table_name} WHERE {condition}"
            cursor.execute(query)
        # Fetch column names
        columns = [col[0] for col in cursor.description]
        # Fetch data
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None
    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()

In [30]:
# Get the the asset classes combined with their parent class ID
table_name = 'AssetClass'
column_names = ''
condition = ''
sql = """SELECT a.`ID`, a.`Title`, a.`Group`, a.`Issuer`, a.`PercentMax`,
CASE WHEN p.`Title` IS NULL THEN a.`Title` ELSE p.`Title` END AS `AssetClassCombined`
FROM `AssetClass` a
LEFT JOIN ( SELECT ac.`ID`, ac.`Title`, ac.`Group`, ac.`PercentMax` FROM `AssetClass` ac
WHERE AssetClassParentID = 0 ) p ON p.ID = a.AssetClassParentID
WHERE AssetClassParentID = 0 AND a.Title != 'Not Assigned'
"""
assetClasses = fetch_data(table_name, column_names,1, sql=sql)
assetClasses

,ID,Title,Group,Issuer,PercentMax,AssetClassCombined
0,1,Cash/Sweep,Not Assigned,None,1.0000,Cash/Sweep
1,2,Certificate of Deposit,Certificates of Deposit,None,0.5000,Certificate of Deposit
2,3,Commercial Paper,Commercial Paper,None,0.3330,Commercial Paper
3,4,US Agencies,US Agencies,None,0.2000,US Agencies
4,7,Money Market,Not Assigned,None,0.2000,Money Market
5,8,Mutual Fund,Not Assigned,None,0.2000,Mutual Fund
6,16,US Treasuries,US Treasuries,None,1.0000,US Treasuries
